<a href="https://colab.research.google.com/github/PhoenixStormJr/Tortoise-tts-fast-longer/blob/main/tortoise_tts_faster2_longer_alpha_update6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [1]:
#@title # Install and restart runtime
import os
def InstallTortoise():
  %cd /content
  #Because the name of tortoise keeps changing every THIRTY SECONDS, I added
  #this variable to help with future changes.
  RepositoryName = 'tortoise-tts-fastest'
  !rm -r /content/{RepositoryName}
  !git clone https://github.com/manmay-nakhashi/tortoise-tts-fastest
  %cd /content/{RepositoryName}
  !pip3 install -r requirements.txt --no-deps
  !pip install transformers==4.29.2 deepspeed pydub
  !pip3 install git+https://github.com/152334H/BigVGAN.git
  !pip3 install -e .


#Checks if the correct version of numpy is installed or not.
if(not os.path.exists("/usr/local/lib/python3.10/dist-packages/numpy-1.23.5.dist-info")):
  #This only checks if a GPU is connected. That is all.
  if len(os.environ["COLAB_GPU"]) > 0:
    InstallTortoise()
  elif "COLAB_TPU_ADDR" in os.environ and os.environ["COLAB_TPU_ADDR"]:
    print("A TPU is connected. A GPU runtime should be used instead. This may (almost definitely will) cause a crash.")
    input1 = input("Would you like to continue without a GPU runtime? (y/n): ")
    if input1 == "y":
      print("Continuing without a GPU")
      InstallTortoise()
    elif(input1 == "n"):
      print("Exiting program")
      raise ValueError("Program closed. No GPU connected")
  else:
    print("There is no GPU (accelerator) connected. A GPU runtime should be used instead. This may (almost definitely will) cause a crash.")
    input1 = input("Would you like to continue without a GPU runtime? (y/n): ")
    if input1 == "y":
      print("Continuing without a GPU")
      InstallTortoise()
    elif(input1 == "n"):
      print("Exiting program")
      raise ValueError("Program closed. No GPU connected")
else:
  print("Tortoise is already installed. No need to run this cell. Please restart the runtime if you haven't already,")

Tortoise is already installed. No need to run this cell. Please restart the runtime if you haven't already,


In [2]:
# Imports used through the rest of the notebook.
RepositoryName = 'tortoise-tts-fastest'
%cd /content/{RepositoryName}
try:
  from numba.np.ufunc import _internal
except SystemError as SystemError1:
  DidntRestartRuntimeError = "initialization of _internal failed without raising an exception"
  if(str(SystemError1) == DidntRestartRuntimeError):
    raise SystemError(f"You did not restart the session/runtime, after installing numpy-1.23.5 The Error message is: \"{SystemError1}\"")

import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython


from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices


# This will download all the models used by Tortoise from the HuggingFace hub.
#This also solves the "Type fp16 is not supported." error, instead returning a REAL error.
try:
  tts = TextToSpeech()
except ValueError as ValueError1:
  NoGPUError = "Type fp16 is not supported."
  if(str(ValueError1) == NoGPUError):
    raise ValueError("No GPU available. Please enable a GPU runtime. Error message: \"Type fp16 is not supported.\"")
  else:
    raise ValueError(f"Unknown solution: {ValueError1}")


/content/tortoise-tts-fastest
[2024-05-07 23:34:47,845] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

...

Time to load transformer_inference op: 83.89711117744446 seconds
[2024-05-07 23:36:59,652] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 1024, 'intermediate_size': 4096, 'heads': 16, 'num_hidden_layers': -1, 'dtype': torch.float16, 'pre_layer_norm': True, 'norm_type': <NormType.LayerNorm: 1>, 'local_rank': -1, 'stochastic_mode': False, 'epsilon': 1e-05, 'mp_size': 1, 'scale_attention': True, 'triangular_masking': True, 'local_attention': False, 'window_size': 1, 'rotary_dim': -1, 'rotate_half': False, 'rotate_every_two': True, 'return_tuple': True, 'mlp_after_attn': True, 'mlp_act_func_type': <ActivationFuncType.GELU: 1>, 'specialized_mode': False, 'training_mp_size': 1, 'bigscience_bloom': False, 'max_out_tokens': 1024, 'min_out_tokens': 1, 'scale_attn_by_inverse_layer_idx': False, 'enable_qkv_quantization': False, 'use_mup': False, 'return_single_tuple': False, 'set_empty_params': False, 'transposed_mode': False, 'use_triton': Fa

Loading extension module transformer_inference...


Done.


Done.


Done.


In [3]:
#@title Display current voices available. (This cell is actually useless....)
#@markdown Tortoise will attempt to mimic voices you provide. It comes pre-packaged
#@markdown with some voices you might recognize.
#@markdown
#@markdown Let's list all the voices available. These are just some random clips I've gathered
#@markdown from the internet as well as a few voices from the training dataset.
#@markdown Feel free to add your own clips to the voices/ folder.
FixNotImplementedError = False # @param {type:"boolean"}
if(FixNotImplementedError):
  import locale
  locale.getpreferredencoding = lambda: "UTF-8"

%cd /content/{RepositoryName}
%ls tortoise/voices
from IPython.display import Audio, display
display(Audio('tortoise/voices/tom/1.wav'))

/content/tortoise-tts-fastest
angie/                emma/     lj/      rainbow/       train_daws/     train_kennard/
applejack/            freeman/  mol/     snakes/        train_dotrice/  train_lescault/
cond_latent_example/  geralt/   myself/  tim_reynolds/  train_dreams/   train_mouse/
daniel/               halle/    pat/     tom/           train_empire/   weaver/
deniro/               jlaw/     pat2/    train_atkins/  train_grace/    william/


In [4]:
#@title Change voice, or upload a new voice.
#@markdown
#@markdown Pick one of the voices from the output above, or choose custom, or random.
#@markdown
#@markdown Optionally, upload and use your own voice (probably what you came here to do). Upload wav files, of any length, as long as the total time is over 12 seconds. You can upload one or more files of any length, as long as the total time is over 12 seconds.
#@markdown
#@markdown Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
#@markdown (Note: random voices can be prone to strange utterances)

voice = 'custom' #@param ["custom", "random", "angie", "applejack", "cond_latent_example", "daniel", "deniro", "emma", "freeman", "geralt", "halle", "jlaw", "lj", "mol", "myself", "pat", "pat2", "rainbow", "snakes", "tim_reynolds", "tom", "train_atkins", "train_daws", "train_dotrice", "train_dreams", "train_empire", "train_grace", "train_kennard", "train_lescault", "train_mouse", "weaver", "william"]
#@markdown If you've already uploaded a voice, but want to delete it and upload a new voice, click this here.
OverWritePastUpload = True #@param {type:"boolean"}
FixNotImplementedError = True # @param {type:"boolean"}


from pydub import AudioSegment
debug = False
def debugPrint(textToPrint):
  if(debug):
    print(textToPrint)
#Create functions.
#This gets the python version
def PythonVersion():
  FirstDirectory = os.listdir('/usr/local/lib/')
  for thing in FirstDirectory:
    if('python' in thing):
      PythonVersionReturn = thing
  return PythonVersionReturn
#This creates a directory, if it doesn't exist. Meaning prevents a crash
def mkdir(directory):
  import os
  folders = directory.split('/')
  path = ''
  for folder in folders:
    path = path + folder+'/'
    if(not os.path.exists(path)):
      os.mkdir(path)
#This installs a pip, if it doesn't exist.
def PipInstall(PipToInstall):
  if(not os.path.exists(f'/usr/local/lib/{PythonVersion()}/dist-packages/{PipToInstall}')):
    !pip install {PipToInstall}
def mergeAudioFiles(inputFiles, outputFile):
  #but if you insist on finding out how this works, I will explain it.
  #This selects the first audio file from the list, and adds it to combined_sounds. which is currently empty
  combined_sounds = AudioSegment.from_wav(f"{inputFiles[0]}")
  #This is a tracker variable to track which file we are on
  fileNumber = 1
  #This is a loop which will go through every single file in the input files
  while(fileNumber < len(inputFiles)):
    #This adds the next sound file in the list to the combined_sounds variable
    combined_sounds = combined_sounds + AudioSegment.from_wav(f"{inputFiles[fileNumber]}")
    #This increases the file in position by one, and repeats the loop
    fileNumber = fileNumber + 1
  #This exports the combined sounds
  combined_sounds.export(f"{outputFile}", format="wav")
  #This tells the user where the output file was saved to.
  print(f'Output file saved to {os.getcwd()}/{outputFile}')
def SplitAudioBetween(AudioFile,FileName, MinimumSeconds, MaximumSeconds):
  #Get the length of the audio file
  total_in_ms = len(AudioFile)
  #Remove every single group of 10 seconds.
  remaining = total_in_ms - (1000*(MaximumSeconds)*(math.floor(total_in_ms/(1000*MaximumSeconds))-2))
  #This variable will be used at the VERY END to determine how many seconds to split at the end.
  secondsFinal = 0
  #This continues to decrease the times, until the LAST remaining time is between 6-10. It then saves the splitted time to a variable called secondsFinal
  repeat = MaximumSeconds
  while(repeat > MinimumSeconds-1):
    testRemaining = remaining
    while (testRemaining > (MaximumSeconds*1000)-1):
      testRemaining = testRemaining - 1000*repeat
    if((testRemaining < 1000*MaximumSeconds) and (testRemaining > MinimumSeconds*1000)):
      debugPrint(f'testRemaining:{testRemaining}')
      secondsFinal = repeat
      break
    repeat = repeat - 1
  #This function splits the audio into segments. It takes an audio file, the seconds to split in, the starting point, and how many segments to split. The last 2 are optional.
  #def SplitAudio(AudioFile, seconds, starting=0, segments=0):
  #SplitAudio(AudioFile, 10, 0, splits)
  splits = math.floor(total_in_ms/(1000*MaximumSeconds))-2
  debugPrint(f'splits:{splits}')
  seconds = MaximumSeconds
  starting = 0
  segments = splits
  CurrentWorkingDirectory = os.getcwd()
  AllFilesAndFolders = os.listdir(CurrentWorkingDirectory)
  FilesWithFileName = []
  for thing in AllFilesAndFolders:
    if(FileName in thing):
      FilesWithFileName.append(thing)
  NumbersStrings = []
  for thing in FilesWithFileName:
    NumbersStrings.append(thing.replace(FileName,'').replace('.wav',''))
  Numbers = []
  for NumberString in NumbersStrings:
    if(len(NumberString) > 0 and NumberString.isnumeric()):
      Numbers.append(int(NumberString))
  if(len(Numbers) == 0):
    repeat2 = 1
  if(len(Numbers) > 0):
    repeat2 = 0
    for Number in Numbers:
      if(Number > repeat2):
        repeat2 = Number
    repeat2 = repeat2 + 1
  trueSeconds = seconds*1000
  repeat = trueSeconds+starting
  if(segments > 0):
    StopAt = ((segments+1)*trueSeconds)
  else:
    StopAt = 0
  print()
  print()
  print(f'Here are the individual splits of the audio you uploaded:')
  print()
  while(repeat < StopAt):
    debugPrint('FIRST')
    print(f'{repeat-trueSeconds}:{repeat}, seconds:{round((repeat-(repeat-trueSeconds))/1000,3)}')
    newAudio = AudioFile[repeat-trueSeconds:repeat]
    newAudio.export(f'{FileName}{repeat2}.wav', format="wav") #Exports to a wav file in the current path.
    EndedOn = repeat
    repeat2 = repeat2 + 1
    repeat = repeat + trueSeconds
  if(segments == 0):
    debugPrint('SECOND')
    print(f'{EndedOn}:{StopAt}, seconds:{StopAt-EndedOn}, seconds: {round((StopAt-EndedOn)/1000,3)}')
    newAudio = AudioFile[EndedOn:StopAt]
    newAudio.export(f'{FileName}{repeat2}.wav', format="wav") #Exports to a wav file in the current path.
    repeat2 = repeat2 + 1
    repeat = repeat + trueSeconds
  #This will get the first split amount, for 10 seconds split, but leave the last 20-30 seconds unchanged
  #SplitAudio(AudioFile, secondsFinal, StartingPointEnd)
  #This will get the pick up point from above.
  if(splits > 0):
    StartingPointEnd = splits*(1000*MaximumSeconds)
  else:
    StartingPointEnd = 0
  #This will split the audio file, based on the special amount of seconds we decided earlier, but it will go all the way to the end.
  #SplitAudio(AudioFile, secondsFinal, StartingPointEnd)
  #def SplitAudio(AudioFile, seconds, starting=0, segments=0):
  seconds = secondsFinal
  starting = StartingPointEnd
  segments = 0
  CurrentWorkingDirectory = os.getcwd()
  AllFilesAndFolders = os.listdir(CurrentWorkingDirectory)
  FilesWithFileName = []
  for thing in AllFilesAndFolders:
    if(FileName in thing):
      FilesWithFileName.append(thing)
  NumbersStrings = []
  for thing in FilesWithFileName:
    NumbersStrings.append(thing.replace(FileName,'').replace('.wav',''))
  Numbers = []
  for NumberString in NumbersStrings:
    if(len(NumberString) > 0 and NumberString.isnumeric()):
      Numbers.append(int(NumberString))
  if(len(Numbers) == 0):
    repeat2 = 1
  if(len(Numbers) > 0):
    repeat2 = 0
    for Number in Numbers:
      if(Number > repeat2):
        repeat2 = Number
    repeat2 = repeat2 + 1
  trueSeconds = seconds*1000
  repeat = trueSeconds+starting
  debugPrint(f'trueSeconds: {trueSeconds}, starting: {starting}')
  if(segments > 0):
    StopAt = ((segments+1)*trueSeconds)
  else:
    #So here, it wasn't parting the seconds right. The last segment was 0.3 seconds.
    #So it first gets the current cursor, known as "repeat", and puts it in "StopAt"
    #Then StopAt is increasing until fullLength - StopAt > 6,000 and fullLength - StopAt < 10,000
    debugPrint(f'trueSeconds:{trueSeconds}')
    debugPrint(f'repeat:{repeat}')
    StopAt = repeat
    while(len(AudioFile)-StopAt > 10000):
      StopAt = StopAt + trueSeconds
      debugPrint(f'len(AudioFile)-StopAt : {len(AudioFile)-StopAt}')
    debugPrint(f'StopAt:{StopAt}')
  if(repeat > StopAt):
    EndedOn = 0
  while(repeat < StopAt+1):
    debugPrint('THIRD')
    print(f'{repeat-trueSeconds}:{repeat}, seconds:{round((repeat-(repeat-trueSeconds))/1000,3)}')
    newAudio = AudioFile[repeat-trueSeconds:repeat]
    EndedOn = repeat
    newAudio.export(f'{FileName}{repeat2}.wav', format="wav") #Exports to a wav file in the current path.
    repeat2 = repeat2 + 1
    repeat = repeat + trueSeconds
  if(segments == 0):
    debugPrint('FOURTH')
    print(f'{EndedOn}:{len(AudioFile)}, seconds:{round((len(AudioFile)-EndedOn)/1000,3)}')
    newAudio = AudioFile[EndedOn:len(AudioFile)]
    newAudio.export(f'{FileName}{repeat2}.wav', format="wav") #Exports to a wav file in the current path.
    repeat2 = repeat2 + 1
    repeat = repeat + trueSeconds




if(FixNotImplementedError):
  import locale
  locale.getpreferredencoding = lambda: "UTF-8"

if(voice == "custom"):
  if(OverWritePastUpload and os.path.exists(f'/content/{RepositoryName}/tortoise/voices/custom')):
    !rm -r /content/{RepositoryName}/tortoise/voices/custom
  #Import and install requirements.
  from google.colab import files
  import math
  #Create the directory to upload the files to, and go to that directory.
  mkdir("/content/uploaded")
  %cd /content/uploaded
  #Upload the files
  uploaded = files.upload()
  #Get the name of the file uploaded
  FilesUploaded = []
  for key in uploaded.keys():
    FilesUploaded.append(key)
  FileName = "uploadedFull.wav"
  mergeAudioFiles(FilesUploaded,FileName)
  #Rename the file to an actually usable name
  #Create an audio file variable
  AudioFile = AudioSegment.from_wav(FileName)
  #Split that audio file into segments between 6 and 10 seconds, with my custom function.
  SplitAudioBetween(AudioFile, "custom", 6, 10)
  #Get a list of all the files in the directory
  NewSplits = os.listdir('/content/uploaded')
  #Get a list of all the real splits between 6 and 10 seconds.
  realOnes = []
  for item in NewSplits:
    if('custom' in item):
      realOnes.append(item)
  #Make a new directory
  mkdir(f'/content/{RepositoryName}/tortoise/voices/custom')
  #Move all those files to that directory.
  for item in realOnes:
    os.rename(f'/content/uploaded/{item}',f'/content/{RepositoryName}/tortoise/voices/custom/{item}')


if(not voice == "random"):
  voice_samples, conditioning_latents = load_voice(voice)

/content/uploaded


Saving GlideTortoiseTest.wav to GlideTortoiseTest.wav
Output file saved to /content/uploaded/uploadedFull.wav


Here are the individual splits of the audio you uploaded:

0:10000, seconds:10.0
10000:20000, seconds:10.0
20000:30000, seconds:10.0
30000:40000, seconds:10.0
40000:50000, seconds:10.0
50000:60000, seconds:10.0
60000:69000, seconds:9.0
69000:78000, seconds:9.0
78000:85625, seconds:7.625


In [ ]:
#@title Run this cell to ACTUALLY generate audio. You can change the text and run it again, even multiple times, without reinstalling tortoise, and you don't even have to re run the above cells.
#@markdown This is the text that will be spoken.
text = "Hello! This is Tortoise voice cloner! This is meant to be used for entertainment. Not for scams, or malicious intent. Please do not use this software for AI scams, because we are people with lives that you could destroy. Thank you and have fun!" #@param {type:"string"}
#@markdown Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast", "standard", "high_quality (default)"}. See docs in api.py
#Here are the weird presets: single_sample, very_fast, ultra_fast_old
preset = "high_quality" #@param ["ultra_fast", "fast", "standard", "high_quality", "single_sample (experimental)", "very_fast (experimental)", "ultra_fast_old (experimental)"]
if(preset == "single_sample (experimental)"):
  preset = "single_sample"
if(preset == "very_fast (experimental)"):
  preset = "very_fast"
if(preset == "ultra_fast_old (experimental)"):
  preset = "ultra_fast_old"
#@markdown This is the new thing I added. The ability to type more sentences. I HIGHLY reccomend checking this box.
SplitSentences = True #@param {type:"boolean"}





#This converts the text into a format that Tortoise can use, by replacing stuff that tortoise doesn't know how to deal with.
text = text.replace(".),",".")
text = text.replace(". (",". ")
text = text.replace(").",".")
text = text.replace(").",".")
text = text.replace("),",",")
text = text.replace("(",".")
text = text.replace(")",".")
text = text.replace(":.","...")
text = text.replace(".:","...")

text = text.replace("GB","Gigabyte")
text = text.replace("MB","Megabyte")
text = text.replace("KB","Kilobyte")
text = text.replace(" - "," dash ")
text = text.replace("-"," dash ")
text = text.replace(" + "," plus ")
text = text.replace("+"," plus ")
text = text.replace(" / "," slash ")
text = text.replace("/"," slash ")
text = text.replace(" _ "," underscore ")
text = text.replace("_"," underscore ")
text = text.replace("GPT","Jee Pee Tee")
text = text.replace("CPU","See Pee You")
text = text.replace("GPU","Jee Pee You")
text = text.replace("FBI","Eff Bee I")
text = text.replace("CDC","See Dee See")
text = text.replace("NIH","Enn I Aitch")
text = text.replace("MPS","Emm Pee Ess")
text = text.replace("TTS","Tee Tee Ess")
text = text.replace("RTF","Are Tee Eff")
text = text.replace("AI","Aey I")

repeat = 0
while(repeat < 10):
  repeat2 = 0
  while(repeat2 < 10):
    text = text.replace(f"{repeat}.{repeat2}",f"{repeat} point {repeat2}")
    repeat2 = repeat2 + 1
  repeat = repeat + 1




#Ignore this, this only makes merging audio files easier. It's a custom made function
def mergeAudioFiles(inputFiles, outputFile):
  #but if you insist on finding out how this works, I will explain it.
  #This selects the first audio file from the list, and adds it to combined_sounds. which is currently empty
  combined_sounds = AudioSegment.from_wav(f"{inputFiles[0]}")
  #This is a tracker variable to track which file we are on
  fileNumber = 1
  #This is a loop which will go through every single file in the input files
  while(fileNumber < len(inputFiles)):
    #This adds the next sound file in the list to the combined_sounds variable
    combined_sounds = combined_sounds + AudioSegment.from_wav(f"{inputFiles[fileNumber]}")
    #This increases the file in position by one, and repeats the loop
    fileNumber = fileNumber + 1
  #This exports the combined sounds
  combined_sounds.export(f"{outputFile}", format="wav")
  #This tells the user where the output file was saved to.
  print(f'Output file saved to {os.getcwd()}/{outputFile}')
###################################################################################




#Loads it and send it through Tortoise.
if(not SplitSentences):
  if((not voice == "custom") and (not voice == "random")):
    torchaudio.save(f'generated-{voice}.wav', tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset).squeeze(0).cpu(), 24000)
    display(Audio(f'generated-{voice}.wav', autoplay=True))

  if(voice == "random"):
    torchaudio.save(f'generated-{voice}.wav', tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset).squeeze(0).cpu(), 24000)
    display(Audio(f'generated-{voice}.wav', autoplay=True))

  # Generate speech with the custom voice.
  if(voice == "custom"):
    torchaudio.save(f'generated-{voice}.wav', tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset).squeeze(0).cpu(), 24000)
    display(Audio(f'generated-{voice}.wav', autoplay=True))

if(SplitSentences):
  #Since Tortoise is really bad on long audio, this will split the text up into multiple sentences.
  sentences = []
  temporarySentence = ""
  multipleDotsTogether = False
  characterNumber = 0
  while(characterNumber < len(text)):
    temporarySentence = temporarySentence + text[characterNumber]
    if(characterNumber < len(text) -1):
      if(text[characterNumber] == "." and text[characterNumber+1] == "."):
        multipleDotsTogether = True
    if(characterNumber < len(text) -1):
      if(text[characterNumber] == "!" and text[characterNumber+1] == "!"):
        multipleDotsTogether = True
    if(characterNumber < len(text) -1):
      if(text[characterNumber] == "?" and text[characterNumber+1] == "?"):
        multipleDotsTogether = True
    if((text[characterNumber] == ":" or text[characterNumber] == "." or text[characterNumber] == "!" or text[characterNumber] == "?" or text[characterNumber] == "," or text[characterNumber] == ";") and (not multipleDotsTogether)):
      sentences.append(temporarySentence.strip())
      temporarySentence = ""
    multipleDotsTogether = False
    characterNumber = characterNumber + 1
  if(len(temporarySentence.strip()) > 1):
    sentences.append(temporarySentence.strip())



  print("Tortoise split the text into multiple sentences. It will generate one at a time, then combine them together in a single file for you.")
  print("Please be patient, this will take a LONG time. Literally....")
  print("Here are the sentences Tortoise will generate in order:")
  print()
  for sentence in sentences:
    print(f'"{sentence}"')
  print()
  print()
  #This actually generates the audio.
  if((not voice == "custom") and (not voice == "random")):
    sentenceNumber = 1
    inputFiles = []
    for sentence in sentences:
      print(f'Generating: "{sentence}"  {sentenceNumber}/{len(sentences)}')
      torchaudio.save(f'generated-{voice}-{sentenceNumber}.wav', tts.tts_with_preset(sentence, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset).squeeze(0).cpu(), 24000)
      display(Audio(f'generated-{voice}-{sentenceNumber}.wav', autoplay=True))
      inputFiles.append(f'generated-{voice}-{sentenceNumber}.wav')
      sentenceNumber = sentenceNumber + 1
    print(inputFiles)
    print("Merging your generated audio files together.")
    #And this function merges the audio files together.
    mergeAudioFiles(inputFiles,f'generated-{voice}.wav')
    #Finally this displays the generated FULL combined audio.
    display(Audio(f'generated-{voice}.wav', autoplay=True))

  if(voice == "random"):
    sentenceNumber = 1
    inputFiles = []
    for sentence in sentences:
      print(f'Generating: "{sentence}"  {sentenceNumber}/{len(sentences)}')
      torchaudio.save(f'generated-{voice}-{sentenceNumber}.wav', tts.tts_with_preset(sentence, voice_samples=None, conditioning_latents=None, preset=preset).squeeze(0).cpu(), 24000)
      display(Audio(f'generated-{voice}-{sentenceNumber}.wav', autoplay=True))
      inputFiles.append(f'generated-{voice}-{sentenceNumber}.wav')
      sentenceNumber = sentenceNumber + 1
    print(inputFiles)
    print("Merging your generated audio files together.")
    #And this function merges the audio files together.
    mergeAudioFiles(inputFiles,f'generated-{voice}.wav')
    #Finally this displays the generated FULL combined audio.
    display(Audio(f'generated-{voice}.wav', autoplay=True))

  if(voice == "custom"):
    sentenceNumber = 1
    inputFiles = []
    for sentence in sentences:
      print(f'Generating: "{sentence}"  {sentenceNumber}/{len(sentences)}')
      torchaudio.save(f'generated-{voice}-{sentenceNumber}.wav', tts.tts_with_preset(sentence, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset).squeeze(0).cpu(), 24000)
      display(Audio(f'generated-{voice}-{sentenceNumber}.wav', autoplay=True))
      inputFiles.append(f'generated-{voice}-{sentenceNumber}.wav')
      sentenceNumber = sentenceNumber + 1
    print(inputFiles)
    print("Merging your generated audio files together.")
    #And this function merges the audio files together.
    mergeAudioFiles(inputFiles,f'generated-{voice}.wav')
    #Finally this displays the generated FULL combined audio.
    display(Audio(f'generated-{voice}.wav', autoplay=True))


Tortoise split the text into multiple sentences. It will generate one at a time, then combine them together in a single file for you.
Please be patient, this will take a LONG time. Literally....
Here are the sentences Tortoise will generate in order:

"Hello!"
"This is Tortoise voice cloner!"
"This is meant to be used for entertainment."
"Not for scams,"
"or malicious intent."
"Please do not use this software for Aey I scams,"
"because we are people with lives that you could destroy."
"Thank you and have fun!"


Generating: "Hello!"  1/8
mode 0
Generating autoregressive samples..


100%|██████████| 16/16 [00:13<00:00,  1.17it/s]


Computing best candidates using CLVP


100%|██████████| 16/16 [00:05<00:00,  2.75it/s]


Transforming autoregressive outputs into audio..


...

100%|██████████| 16/16 [00:06<00:00,  2.50it/s]


Transforming autoregressive outputs into audio..


  0%|          | 0/400 [00:00<?, ?it/s]

Generating: "Please do not use this software for Aey I scams,"  6/8
mode 0
Generating autoregressive samples..


 25%|██▌       | 4/16 [00:07<00:23,  1.95s/it]

In [ ]:
#@title Combine voices into one.
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:

RunCombo = False #@param {type:"boolean"}
voice1 = "pat" # @param ["custom", "angie", "applejack", "cond_latent_example", "daniel", "deniro", "emma", "freeman", "geralt", "halle", "jlaw", "lj", "mol", "myself", "pat", "pat2", "rainbow", "snakes", "tim_reynolds", "tom", "train_atkins", "train_daws", "train_dotrice", "train_dreams", "train_empire", "train_grace", "train_kennard", "train_lescault", "train_mouse", "weaver", "william"] {allow-input: true}
voice2 = "william" # @param ["custom", "angie", "applejack", "cond_latent_example", "daniel", "deniro", "emma", "freeman", "geralt", "halle", "jlaw", "lj", "mol", "myself", "pat", "pat2", "rainbow", "snakes", "tim_reynolds", "tom", "train_atkins", "train_daws", "train_dotrice", "train_dreams", "train_empire", "train_grace", "train_kennard", "train_lescault", "train_mouse", "weaver", "william"] {allow-input: true}
text = "They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to."#@param {type:"string"}
preset = "high_quality" #@param ["ultra_fast", "fast", "standard", "high_quality"]
SplitSentences = True #@param {type:"boolean"}

def mergeAudioFiles(inputFiles, outputFile):
  #but if you insist on finding out how this works, I will explain it.
  #This selects the first audio file from the list, and adds it to combined_sounds. which is currently empty
  combined_sounds = AudioSegment.from_wav(f"{inputFiles[0]}")
  #This is a tracker variable to track which file we are on
  fileNumber = 1
  #This is a loop which will go through every single file in the input files
  while(fileNumber < len(inputFiles)):
    #This adds the next sound file in the list to the combined_sounds variable
    combined_sounds = combined_sounds + AudioSegment.from_wav(f"{inputFiles[fileNumber]}")
    #This increases the file in position by one, and repeats the loop
    fileNumber = fileNumber + 1
  #This exports the combined sounds
  combined_sounds.export(f"{outputFile}", format="wav")
  #This tells the user where the output file was saved to.
  print(f'Output file saved to {os.getcwd()}/{outputFile}')




if(RunCombo):
  if(not SplitSentences):
    voice_samples, conditioning_latents = load_voices([voice1, voice2])
    torchaudio.save('captain_kirkard.wav', tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset).squeeze(0).cpu(), 24000)
    display(Audio(f'captain_kirkard.wav', autoplay=True))
  if(SplitSentences):
    voice_samples, conditioning_latents = load_voices([voice1, voice2])
    #Since Tortoise is really bad on long audio, this will split the text up into multiple sentences.
    sentences = []
    temporarySentence = ""
    multipleDotsTogether = False
    characterNumber = 0
    while(characterNumber < len(text)):
      temporarySentence = temporarySentence + text[characterNumber]
      if(characterNumber < len(text) -1):
        if(text[characterNumber] == "." and text[characterNumber+1] == "."):
          multipleDotsTogether = True
      if(characterNumber < len(text) -1):
        if(text[characterNumber] == "!" and text[characterNumber+1] == "!"):
          multipleDotsTogether = True
      if(characterNumber < len(text) -1):
        if(text[characterNumber] == "?" and text[characterNumber+1] == "?"):
          multipleDotsTogether = True
      if((text[characterNumber] == ":" or text[characterNumber] == "." or text[characterNumber] == "!" or text[characterNumber] == "?" or text[characterNumber] == "," or text[characterNumber] == ";") and (not multipleDotsTogether)):
        sentences.append(temporarySentence.strip())
        temporarySentence = ""
      multipleDotsTogether = False
      characterNumber = characterNumber + 1
    if(len(temporarySentence.strip()) > 1):
      sentences.append(temporarySentence.strip())

    print("Tortoise split the text into multiple sentences. It will generate one at a time, then combine them together in a single file for you.")
    print("Please be patient, this will take a LONG time. Literally....")
    print("Here are the sentences Tortoise will generate in order:")
    print()
    for sentence in sentences:
      print(f'"{sentence}"')
    print()
    print()

    sentenceNumber = 1
    inputFiles = []
    for sentence in sentences:
      print(f'Generating: "{sentence}"  {sentenceNumber}/{len(sentences)}')
      torchaudio.save(f'captain_kirkard-{sentenceNumber}.wav', tts.tts_with_preset(sentence, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset).squeeze(0).cpu(), 24000)
      display(Audio(f'captain_kirkard-{sentenceNumber}.wav', autoplay=True))
      inputFiles.append(f'captain_kirkard-{sentenceNumber}.wav')
      sentenceNumber = sentenceNumber + 1
    print(inputFiles)
    print("Merging your generated audio files together.")
    #And this function merges the audio files together.
    mergeAudioFiles(inputFiles,f'captain_kirkard.wav')
    #Finally this displays the generated FULL combined audio.
    display(Audio(f'captain_kirkard.wav', autoplay=True))


In [ ]:
#@title Just debugging
LaunchDebugTerminal = False #@param {type:"boolean"}
if(LaunchDebugTerminal):
  !pip install colab-xterm
  %load_ext colabxterm
  %xterm